In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from time import sleep
options = webdriver.ChromeOptions()
options.add_argument('--headless')
service = Service(ChromeDriverManager().install())
# Inicializamos el WebDriver
driver = webdriver.Chrome()

# Configuramos el tiempo de espera implícito a 10 segundos
driver.implicitly_wait(10)

# URL de la web de hoteles
# En este caso la url ya lleva las selecciones de fecha y disponibilidad
url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

# Abrimos la url
driver.get(url_hoteles)
#driver.maximize_window()
sleep(3)  # Pausa para permitir la carga completa de la página
try:
    # Intentar encontrar y hacer clic en el botón de aceptar cookies
    boton_cookies = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    boton_cookies.click()
    print("Cookies aceptadas.")
except NoSuchElementException:
    print("No se encontraron cookies para aceptar.")

# Encuentra todos los bloques de información de los hoteles en la página
hotel_blocks = driver.find_elements(By.CLASS_NAME, "hotelblock__content")

# Lista vacía para almacenar la información de los hoteles
hoteles_info = []

# Itera sobre cada bloque de hotel y extrae la información
for hotel in hotel_blocks:
    nombre_hotel = (hotel.find_element(By.CLASS_NAME, "title").text).split('\n')[0] # split('\n')[0] para obtener solo el nombre del hotel
    estrellas = (hotel.find_element(By.CLASS_NAME, "ratings__score").text).split("/")[0] # obtener solo el número de estrellas
    # Como hay hoteles con precios para socios, queremos obtener el precio por defecto
    try: # intentamos obtener el precio por defecto
        precio = hotel.find_element(By.CLASS_NAME, "rate-details__price2__first-price.rate-details__price--default .booking-price__number.mcp-price-number").text
        
    except: # si solo hay un precio nos devuelve el único disponible
        precio = hotel.find_element(By.CLASS_NAME, "rate-details__price1__first-price.rate-details__price--default .booking-price__number.mcp-price-number").text
        
# añadimos la información a la lista
    hoteles_info.append({
        "nombre": nombre_hotel,
        "estrellas": estrellas,
        "precio": precio
    })
# Cerrar el navegador
driver.quit()
#driver.close()



Cookies aceptadas.


In [20]:
# Crear un dataframe con la información obtenida
df_hoteles_competencia2 = pd.DataFrame(hoteles_info)
df_hoteles_competencia2.to_csv('hoteles_competencia2.csv', index=False)